<a href="https://colab.research.google.com/github/dipucriodigital/ciencia-de-dados-e-analytics/blob/main/advanced-analytics/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Similaridade de palavras em contexto - Word2Vec

Neste exercício vamos calcular a similaridade entre palavras dado um contexto apresentado. Neste caso, vamos analisar textos de machado de assis e ver quais termos aparecem mais próximo dado o contexto da narrativa.

Para isso vamos utilizar 3 bibliotecas para processamento de texto: NLTK, Spacy e Gensim.

*Adaptadado de: https://www.kaggle.com/code/pierremegret/gensim-word2vec-tutorial/notebook

In [ ]:
import re  # For preprocessing
import pandas as pd  # For data handling
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

import spacy  # For preprocessing

import logging  # Setting up the loggings to monitor gensim
logging.basicConfig(format="%(levelname)s - %(asctime)s: %(message)s", datefmt= '%H:%M:%S', level=logging.INFO)

Vamos fazer o download do pipeline para processamento de texto em português da Spacy.

In [ ]:
!python3 -m spacy download pt_core_news_sm
from spacy.lang.pt.examples import sentences 


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 19.7 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


Vamos fazer o download da base de texto das obras de machado de assis. (incluso na nltk)

In [ ]:
import nltk
nltk.download('machado')
from nltk.corpus import machado

machado.fileids()

[nltk_data] Downloading package machado to /root/nltk_data...
[nltk_data]   Package machado is already up-to-date!


['contos/macn001.txt',
 'contos/macn002.txt',
 'contos/macn003.txt',
 'contos/macn004.txt',
 'contos/macn005.txt',
 'contos/macn006.txt',
 'contos/macn007.txt',
 'contos/macn008.txt',
 'contos/macn009.txt',
 'contos/macn010.txt',
 'contos/macn011.txt',
 'contos/macn012.txt',
 'contos/macn013.txt',
 'contos/macn014.txt',
 'contos/macn015.txt',
 'contos/macn016.txt',
 'contos/macn017.txt',
 'contos/macn018.txt',
 'contos/macn019.txt',
 'contos/macn020.txt',
 'contos/macn021.txt',
 'contos/macn022.txt',
 'contos/macn023.txt',
 'contos/macn024.txt',
 'contos/macn025.txt',
 'contos/macn026.txt',
 'contos/macn027.txt',
 'contos/macn028.txt',
 'contos/macn029.txt',
 'contos/macn030.txt',
 'contos/macn031.txt',
 'contos/macn032.txt',
 'contos/macn033.txt',
 'contos/macn034.txt',
 'contos/macn035.txt',
 'contos/macn036.txt',
 'contos/macn037.txt',
 'contos/macn038.txt',
 'contos/macn039.txt',
 'contos/macn040.txt',
 'contos/macn041.txt',
 'contos/macn042.txt',
 'contos/macn043.txt',
 'contos/ma

Antes de começarmos a análise precisamos extrair caracteres não alfa-numéricos, stop-words e reduzir as palavras ao seu radical. Isto é uma etapa essencial no pré-processamento de dados.

In [ ]:

raw_text = machado.raw('romance/marm05.txt') #memórias bras cubas

spacy.cli.download("pt_core_news_sm")
nlp = spacy.load('pt_core_news_sm', disable=['ner', 'parser']) # disabling Named Entity Recognition for speed

def cleaning(doc):
    # Lemmatizes and removes stopwords
    # doc needs to be a spacy Doc object
    txt = [token.lemma_ for token in doc if not token.is_stop]
    # Word2Vec uses context words to learn the vector representation of a target word,
    # if a sentence is only one or two words long,
    # the benefit for the training is very small
    if len(txt) > 2:
        return ' '.join(txt)

new_str = re.sub(r'[^\w\s]', '', raw_text).lower() #remover caracteres não-alfa numéricos
t = time()
doc = nlp(new_str)
txt = cleaning(doc)

print('Time to clean up everything: {} mins'.format(round((time() - t) / 60, 2)))

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
Time to clean up everything: 0.06 mins


In [ ]:
txt = txt.split('\n')
txt

['romancer memória póstum brás cuba 1880 ',
 '',
 ' memória póstum ',
 ' brá cuba ',
 '',
 ' textofonte ',
 ' obra completo machado ',
 ' assi ',
 ' rio ',
 ' janeiro editora aguilar 1994 ',
 '',
 ' publicar originalmente ',
 ' folhetim março 1880 revista brasileiro ',
 '',
 ' verme ',
 ' ',
 ' roeu fria ',
 ' carne ',
 ' cadáver ',
 '',
 ' dedico ',
 ' saudoso lembrança ',
 '',
 ' ',
 ' memória ',
 ' póstuma ',
 '',
 ' prólogo ',
 ' edição ',
 '',
 ' edição de este ',
 ' memória póstum brá cuba fazer pedaço revista ',
 ' brasileira ano 1880 posta livro corrigi texto ',
 ' lugar rever edição emendei ',
 ' algum suprimi dúzia linha composta sair ',
 ' novamente luz obra algum benevolência encontrar ',
 ' público ',
 '',
 ' capistrano abrer noticiar ',
 ' publicação livro perguntar memória póstum brás cuba ',
 ' romance Macedo Soares carta escrever recordar ',
 ' amigamente viagem terra responder defunto ',
 ' brás cuba leitor ver ver prólogo de ele adiante ',
 ' romancer ',
 ' explicar 

Vamos criar um dataframe com o texto processado

In [ ]:
data_dict = {'clean': txt}
df_clean = pd.DataFrame(data_dict)
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(5717, 1)

In [ ]:
!pip install gensim
from gensim.models.phrases import Phrases, Phraser

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
sent = [row.split() for row in df_clean['clean']]
sent

[['romancer', 'memória', 'póstum', 'brás', 'cuba', '1880'],
 [],
 ['memória', 'póstum'],
 ['brá', 'cuba'],
 ['textofonte'],
 ['obra', 'completo', 'machado'],
 ['assi'],
 ['rio'],
 ['janeiro', 'editora', 'aguilar', '1994'],
 ['publicar', 'originalmente'],
 ['folhetim', 'março', '1880', 'revista', 'brasileiro'],
 ['verme'],
 [],
 ['roeu', 'fria'],
 ['carne'],
 ['cadáver'],
 ['dedico'],
 ['saudoso', 'lembrança'],
 ['memória'],
 ['póstuma'],
 ['prólogo'],
 ['edição'],
 ['edição', 'de', 'este'],
 ['memória', 'póstum', 'brá', 'cuba', 'fazer', 'pedaço', 'revista'],
 ['brasileira', 'ano', '1880', 'posta', 'livro', 'corrigi', 'texto'],
 ['lugar', 'rever', 'edição', 'emendei'],
 ['algum', 'suprimi', 'dúzia', 'linha', 'composta', 'sair'],
 ['novamente', 'luz', 'obra', 'algum', 'benevolência', 'encontrar'],
 ['público'],
 ['capistrano', 'abrer', 'noticiar'],
 ['publicação', 'livro', 'perguntar', 'memória', 'póstum', 'brás', 'cuba'],
 ['romance', 'Macedo', 'Soares', 'carta', 'escrever', 'recordar']

Vamos processar o texto para considerar bigramas. Esta etapa é importante para identificar palavras compostas como "Brás Cubas"

In [ ]:
phrases = Phrases(sent,min_count=1, progress_per=10)

In [ ]:
bigram = Phraser(phrases)
sentences = bigram[sent]
sentences

Vamos também ver quais as palavras mais frequentes no texto após remover stop-words

In [ ]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

8741

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['dizer',
 'capítulo',
 'ter',
 'haver',
 'ser',
 'homem',
 'ir',
 'algum',
 'olho',
 'outro']

#Treinando um modelo Word2Vec

Vamos agora treinar um modelo para retornar as palavras mais próximas dado o contexto da escrita deste conto de machado de assis.

Word2Vec():
Nesta primeira etapa, configuramos os parâmetros do modelo.

.build_vocab():
Aqui construímos o vocabulário a partir de uma sequência de frases e assim inicializamos o modelo.

.Train():
Finalmente, treinamos o modelo.


In [ ]:
import multiprocessing

from gensim.models import Word2Vec
cores = multiprocessing.cpu_count()

#Estes são os parâmetros do modelo segundo a referência da bilbioteca Gensim:

*   min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)
*   window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)
*   size = int - Dimensionality of the feature vectors. - (50, 300)
*   sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)
*   alpha = float - The initial learning rate - (0.01, 0.05)
*   min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
*   negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
*  workers = int - Use these many worker threads to train the model (=faster training with multicore machines)



In [ ]:
w2v_model = Word2Vec(min_count=3,
                     window=5,
                     size=300,
                     sample=1e-3, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)
t = time()

#Vamos contruir o vocabulário
w2v_model.build_vocab(sentences)

print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.01 mins


In [ ]:
t = time()

w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=20, report_delay=1)

print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.06 mins


In [ ]:
w2v_model.init_sims(replace=True)

#Vamos agora verificar a similaridade entre algumas palavras

In [ ]:
w2v_model.wv.most_similar(positive=["brás"])

[('cocheiro', 0.9999237060546875),
 ('egoísta', 0.9999216198921204),
 ('descansar', 0.9999199509620667),
 ('inicial', 0.9999192953109741),
 ('contemplação', 0.9999191761016846),
 ('interior', 0.9999179840087891),
 ('mole', 0.9999179840087891),
 ('substância', 0.9999163746833801),
 ('finado', 0.9999162554740906),
 ('tanoaria', 0.9999161958694458)]

In [ ]:
w2v_model.wv.most_similar(negative=["homem"])

[('___', -0.6152580380439758),
 ('ver', -0.9986587762832642),
 ('cuba', -0.9987366795539856),
 ('mão', -0.998762845993042),
 ('crer', -0.9988263845443726),
 ('velho', -0.9988966584205627),
 ('beijo', -0.9988994598388672),
 ('passar', -0.9989227056503296),
 ('mesmo', -0.9989330768585205),
 ('marcela', -0.998935341835022)]

#Exercício 1

Agora que você já apredeu a como processar textos e verificar a similaridade entre palavras, verifique quais parâmetros podem de fato alterar os resultdos do modelo. Como podemos melhorar os resultados? E se combinarmos mais de um texto? Qual o resultado?

#Desafio: 
Como podemos fazer analogias com as palavras pesquisadas? Ex: mulher + brás - homem = ?